In [2]:
import pandas as pd
import numpy as np

from time import time
from string import digits
import random
import threading

from geopy import Point
from geopy import Nominatim
from postal.parser import parse_address
from postal.expand import expand_address
import googlemaps
import warnings
# from googlemaps_api_key import key 

# key = key.replace('†','')
gmaps = googlemaps.Client(key=key)
# https://console.cloud.google.com/apis/credentials?project=la-api-key


 API queries_quota: 60 



In [3]:
cd 

/Users/karatechop


In [4]:
cd /Users/karatechop/Documents/GitHub/senior-thesis-pretextual-stops/Data/Formatted with YOE

/Users/karatechop/Documents/GitHub/senior-thesis-pretextual-stops/Data/Formatted with YOE


In [4]:
ls

ALL-BPD-AB953-DUSK-LOC.csv  ALL-OAKLAND_PD-AB953.csv
ALL-BPD-AB953-DUSK.csv      ALL-SDPD-AB953-DUSK.csv
ALL-BPD-AB953.csv           ALL-SDPD-AB953.csv
ALL-MARIN-AB953-DUSK.csv    ALL-SFPD-AB953-DUSK.csv
ALL-MARIN-AB953.csv         ALL-SFPD-AB953.csv
ALL-OAKLAND-AB953-DUSK.csv  sf_zips.csv


## Function Setup

In [5]:
# def find_lat_lon_GMV3_debuggARR(city,address):
#     bounds = gmaps.geocode(city)[0]['geometry']['bounds']
    
#     if address != None: 
#         try:
#             return gmaps.geocode(address,bounds=bounds)

#         except:
#             print(f'Google maps failed for {address}')
          
#             pass

In [8]:
def find_lat_lon_GMV3(city,df,location_col='location'):
    df['neighborhood'],df['zip'],df['coords'] = None,None,None
    gmv3_total_start = time()
    counter = 0
    
    global failed_addresses, failed_counter
    failed_addresses = []
    failed_counter = 0
    
    bounds = gmaps.geocode(city)[0]['geometry']['bounds']
    
    for address in df[location_col].unique(): 
        if address != None and 'freeway' not in address: 
            try:
                result = gmaps.geocode(address,bounds=bounds)
                
                df.loc[df[location_col]==address,'gmap address'] = result[0]['formatted_address']
                df.loc[df[location_col]==address,'neighborhood'] = result[0]['address_components'][1]['long_name']
                df.loc[df[location_col]==address,'zip'] = result[0]['address_components'][-1]['long_name']
                df.loc[df[location_col]==address,'coords'] = str(result[0]['geometry']['location']['lat']) + ', ' +\
                                                             str(result[0]['geometry']['location']['lng'])
            
            except:
#                 print(f'Google maps failed for {address}')
                
                failed_addresses.append(address)
                failed_counter+=1
                
                pass
        
        counter+=1
        if counter%500==0:
            time_passed = round((time()-gmv3_total_start)/60,2)
            print(f'Counter is at {counter} for {city}. {time_passed} minutes have passed.')

## WAVE 1

In [7]:
SFPD = pd.read_csv('ALL-SFPD-AB953-DUSK.csv')
SFPD = SFPD[SFPD['itp']==1]

SFPD.shape

(24107, 14)

In [8]:
SFPD = SFPD.sort_values(by='location')
SFPD['location'] = SFPD['location'].str.lower()

SFPD.loc[(SFPD['location'].str.contains('upper')) |\
                   (SFPD['location'].str.contains('lower')) |\
                   (SFPD['location'].str.contains('sfia')) |\
                   (SFPD['location'].str.contains('terminal')) |\
                   (SFPD['location'].str.contains('sfo')), 'location'] = 'SFO'

SFPD.loc[(SFPD['location'].str.contains('nb')) |\
                   (SFPD['location'].str.contains('northbound')) |\
                   (SFPD['location'].str.contains('sb')) |\
                   (SFPD['location'].str.contains('southbound')) |\
                   (SFPD['location'].str.contains('101')) |\
                   (SFPD['location'].str.contains('ramp')), 'location'] = 'hwy 101'

SFPD['location'] = SFPD['location'].str.replace('/',' & ')
SFPD['location'] = SFPD['location'].str.replace(' / ',' & ')
SFPD['location'] = SFPD['location'].str.replace('/ ',' & ')
SFPD['location'] = SFPD['location'].str.replace(' @ ',' & ')
SFPD['location'] = SFPD['location'].str.replace(' and ',' & ')
SFPD['location'] = SFPD['location'].str.replace(' st. ',' street ')
SFPD['location'] = SFPD['location'].str.replace(' st ',' street ')



SFPD['location'] = SFPD['location'].str.replace('block of',' ')
SFPD['location'] = SFPD['location'].str.replace(' of ','')
SFPD['location'] = SFPD['location'].str.replace('blk',' ')
SFPD['location'] = SFPD['location'].str.replace('block',' ')
SFPD['location'] = SFPD['location'].str.replace('0-100','0')
SFPD['location'] = SFPD['location'].str.replace('0-99','0')


SFPD['location'] = SFPD['location'].str.replace('01st','1st')
SFPD['location'] = SFPD['location'].str.replace('02nd','2nd')
SFPD['location'] = SFPD['location'].str.replace('03rd','3rd')
SFPD['location'] = SFPD['location'].str.replace('04th','4th')
SFPD['location'] = SFPD['location'].str.replace('05th','5th')
SFPD['location'] = SFPD['location'].str.replace('06th','6th')
SFPD['location'] = SFPD['location'].str.replace('07th','7th')
SFPD['location'] = SFPD['location'].str.replace('08th','8th')
SFPD['location'] = SFPD['location'].str.replace('09th','9th')

/var/folders/td/yc8lq2qx2ks3ck292tbf6wtm0000gn/T/ipykernel_11576/3011667247.py:22: FutureWarning: The default value of regex will change from True to False in a future version.
  SFPD['location'] = SFPD['location'].str.replace(' st. ',' street ')


In [10]:
len(SFPD['location'].unique())

10660

In [17]:
warnings.filterwarnings("ignore") 

SFPD1 = SFPD.iloc[:int(SFPD.shape[0]/3),:]
SFPD2 = SFPD.iloc[int(SFPD.shape[0]/3):int(SFPD.shape[0]/3)*2,:]
SFPD3 = SFPD.iloc[int(SFPD.shape[0]/3)*2:,:]


if __name__ =="__main__":
    t1 = threading.Thread(target=find_lat_lon_GMV3,args=('SAN FRANCISCO',SFPD1))
    t2 = threading.Thread(target=find_lat_lon_GMV3,args=('SAN FRANCISCO',SFPD2))
    t3 = threading.Thread(target=find_lat_lon_GMV3,args=('SAN FRANCISCO',SFPD3))
    
    t1.start()
    t2.start()
    t3.start()
 
    t1.join()
    t2.join()
    t3.join()
 
    # both threads completely executed
    print("\n\n\All threads are done!\n\n")

Counter is at 500 for SAN FRANCISCO. 0.88 minutes have passed.
Counter is at 500 for SAN FRANCISCO. 0.93 minutes have passed.
Counter is at 500 for SAN FRANCISCO. 0.95 minutes have passed.
Counter is at 1000 for SAN FRANCISCO. 1.76 minutes have passed.
Counter is at 1000 for SAN FRANCISCO. 1.82 minutes have passed.
Counter is at 1000 for SAN FRANCISCO. 1.93 minutes have passed.
Counter is at 1500 for SAN FRANCISCO. 2.75 minutes have passed.
Counter is at 1500 for SAN FRANCISCO. 2.82 minutes have passed.
Counter is at 1500 for SAN FRANCISCO. 2.91 minutes have passed.
Counter is at 2000 for SAN FRANCISCO. 3.72 minutes have passed.
Counter is at 2000 for SAN FRANCISCO. 3.8 minutes have passed.
Counter is at 2000 for SAN FRANCISCO. 3.89 minutes have passed.
Counter is at 2500 for SAN FRANCISCO. 4.72 minutes have passed.
Counter is at 2500 for SAN FRANCISCO. 4.83 minutes have passed.
Counter is at 2500 for SAN FRANCISCO. 4.92 minutes have passed.
Counter is at 3000 for SAN FRANCISCO. 5.75 m

In [18]:
SFPD_lat_lon = pd.concat([SFPD1,SFPD2,SFPD3])
SFPD_lat_lon.shape

(24107, 18)

In [19]:
zips = pd.read_csv('sf_zips.csv')
zips['ZIP Code'] = zips['ZIP Code'].str[-5:]
sf_zips = [i for i in zips['ZIP Code']]

SFPD_lat_lon.loc[(SFPD_lat_lon['zip'].str.len()<5) |
                 (SFPD_lat_lon['zip'].str.len()>5) |
                 (SFPD_lat_lon['zip'].str[0]!='9'),'zip'] = SFPD_lat_lon['gmap address'].str[-10:-5]

SFPD_lat_lon = SFPD_lat_lon[SFPD_lat_lon['zip'].str[0]=='9']
SFPD_lat_lon = SFPD_lat_lon[SFPD_lat_lon['zip'].isin(sf_zips)]
SFPD_lat_lon.shape

(21324, 18)

In [ ]:
# SFPD_lat_lon['zip'].value_counts().rename_axis('location').reset_index(name='count')

In [20]:
SFPD_lat_lon.to_csv('ALL-SFPD-AB953-DUSK-LOC.csv',index=False)

## WAVES 2-4

In [22]:
BPD = pd.read_csv('ALL-BPD-AB953-DUSK.csv')
BPD = BPD[BPD['itp']==1]
BPD.shape

(1090, 14)

In [23]:
BPD = BPD.sort_values(by='location')
BPD['location'] = BPD['location'].str.lower()


BPD['location'] = BPD['location'].str.replace('/',' & ')
BPD['location'] = BPD['location'].str.replace(' / ',' & ')
BPD['location'] = BPD['location'].str.replace('/ ',' & ')
BPD['location'] = BPD['location'].str.replace(' @ ',' & ')
BPD['location'] = BPD['location'].str.replace(' and ',' & ')
BPD['location'] = BPD['location'].str.replace(' st.',' street ')

BPD['location'] = BPD['location'].str.replace('block of','')
BPD['location'] = BPD['location'].str.replace(' of ','')


In [24]:
warnings.filterwarnings("ignore") 

BPD1 = BPD.iloc[:int(BPD.shape[0]/4),:]
BPD2 = BPD.iloc[int(BPD.shape[0]/4):int(BPD.shape[0]/4)*2,:]
BPD3 = BPD.iloc[int(BPD.shape[0]/4)*2:int(BPD.shape[0]/4)*3,:]
BPD4 = BPD.iloc[int(BPD.shape[0]/4)*3:,:]

if __name__ =="__main__":
    t1 = threading.Thread(target=find_lat_lon_GMV3,args=('BERKELEY, CA',BPD1))
    t2 = threading.Thread(target=find_lat_lon_GMV3,args=('BERKELEY, CA',BPD2))
    t3 = threading.Thread(target=find_lat_lon_GMV3,args=('BERKELEY, CA',BPD3))
    t4 = threading.Thread(target=find_lat_lon_GMV3,args=('BERKELEY, CA',BPD4))
    
    t1.start()
    t2.start()
    t3.start()
    t4.start()
 
    t1.join()
    t2.join()
    t3.join()
    t4.join()
 
    # both threads completely executed
    print("\n\n\All threads are done!\n\n")
    
BPD_lat_lon = pd.concat([BPD1,BPD2,BPD3,BPD4])
BPD_lat_lon.shape



\All threads are done!




(1090, 18)

In [39]:
BPD_lat_lon = BPD_lat_lon.dropna()
BPD_lat_lon.shape

(1059, 18)

In [40]:
# berkeley_zips = ['94608','94702','94703','94704','94705','94706']

In [41]:
BPD_lat_lon.to_csv('ALL-BPD-AB953-DUSK-LOC.csv',index=False)

In [5]:
MARIN = pd.read_csv('ALL-MARIN-AB953-DUSK.csv')
MARIN = MARIN[MARIN['itp']==1]
MARIN.shape

(795, 15)

In [6]:
MARIN = MARIN.sort_values(by='location')
MARIN['location'] = MARIN['location'].str.lower()

MARIN.loc[(MARIN['location'].str.contains(' nb ')) |\
          (MARIN['location'].str.contains('northbound')) |\
          (MARIN['location'].str.contains(' sb ')) |\
          (MARIN['location'].str.contains(' eb ')) |\
          (MARIN['location'].str.contains(' wb ')) |\
          (MARIN['location'].str.contains('southbound')) |\
          (MARIN['location'].str.contains('ramp')) |\
          (MARIN['location'].str.contains('overpass')), 'location'] = 'CA-37'

MARIN.loc[(MARIN['location'].str.contains('101')) |\
          (MARIN['location'].str.contains('hwy')) |\
          (MARIN['location'].str.contains('highway')), 'location'] = 'hwy 101'




MARIN['location'] = MARIN['location'].str.replace(' n ',' north ')
MARIN['location'] = MARIN['location'].str.replace('n. ',' north ')
MARIN['location'] = MARIN['location'].str.replace(' e ',' east ')
MARIN['location'] = MARIN['location'].str.replace(' s ',' south ')
MARIN['location'] = MARIN['location'].str.replace(' w ',' west ')


MARIN['location'] = MARIN['location'].str.replace('/',' & ')
MARIN['location'] = MARIN['location'].str.replace(' / ',' & ')
MARIN['location'] = MARIN['location'].str.replace('/ ',' & ')
MARIN['location'] = MARIN['location'].str.replace('@',' & ')
MARIN['location'] = MARIN['location'].str.replace(' and ',' & ')
MARIN['location'] = MARIN['location'].str.replace(' st ',' street ')

# MARIN['location'] = MARIN['location'].str.replace('block of','')
MARIN['location'] = MARIN['location'].str.replace('blk','')
# MARIN['location'] = MARIN['location'].str.replace(' of ','')

MARIN['location'] = MARIN['location'].str.replace('\xa0','')
MARIN['location'] = MARIN['location'].str.replace('00uth',' south ')
MARIN['location'] = MARIN['location'].str.replace('00r','')
MARIN['location'] = MARIN['location'].str.replace('1 sr','marin')
MARIN['location'] = MARIN['location'].str.replace('bridge a','')
MARIN['location'] = MARIN['location'].str.replace('ander00n','anderson')
MARIN['location'] = MARIN['location'].str.replace('wil00n','wilson')
MARIN['location'] = MARIN['location'].str.replace('ander00n','anderson')

/var/folders/td/yc8lq2qx2ks3ck292tbf6wtm0000gn/T/ipykernel_39524/295272892.py:21: FutureWarning: The default value of regex will change from True to False in a future version.
  MARIN['location'] = MARIN['location'].str.replace('n. ',' north ')


In [9]:
warnings.filterwarnings("ignore") 

MARIN1 = MARIN.iloc[:int(MARIN.shape[0]/4),:]
MARIN2 = MARIN.iloc[int(MARIN.shape[0]/4):int(MARIN.shape[0]/4)*2,:]
MARIN3 = MARIN.iloc[int(MARIN.shape[0]/4)*2:int(MARIN.shape[0]/4)*3,:]
MARIN4 = MARIN.iloc[int(MARIN.shape[0]/4)*3:,:]

if __name__ =="__main__":
    t1 = threading.Thread(target=find_lat_lon_GMV3,args=('MARIN, CA',MARIN1))
    t2 = threading.Thread(target=find_lat_lon_GMV3,args=('MARIN, CA',MARIN2))
    t3 = threading.Thread(target=find_lat_lon_GMV3,args=('MARIN, CA',MARIN3))
    t4 = threading.Thread(target=find_lat_lon_GMV3,args=('MARIN, CA',MARIN4))
    
    t1.start()
    t2.start()
    t3.start()
    t4.start()
 
    t1.join()
    t2.join()
    t3.join()
    t4.join()
 
    # both threads completely executed
    print("\n\n\All threads are done!\n\n")
    
MARIN_lat_lon = pd.concat([MARIN1,MARIN2,MARIN3,MARIN4])
MARIN_lat_lon.shape



\All threads are done!




(795, 19)

In [13]:
MARIN_lat_lon = pd.concat([MARIN1,MARIN2,MARIN3,MARIN4])
MARIN_lat_lon
# MARIN_lat_lon.isna().sum()

,date,time,race,type,race perceived prior,experience,location,sunsettime,dusktime,mins_sunset,mins_dusk,mins_stop_time,mins_before_dusk,itp,dark,neighborhood,zip,coords,gmap address
1845,2021-11-09 20:06:00,1900-01-01 20:06:00,black/african american,2_Equipment violation,No,6.0,sfd & laurel grove,17:03:14,17:31:33,1023.0,1052.0,1206.0,-154.0,1,1,None,None,None,NaN
910,2021-03-31 20:33:00,1900-01-01 20:33:00,white,2_Equipment violation,NaN,21.0,hwy 101,19:33:22,20:00:28,1173.0,1200.0,1233.0,-33.0,1,1,United States,United States,"41.3632465, -123.9894411","US-101, United States"
2019,2021-12-11 19:47:00,1900-01-01 19:47:00,white,2_Equipment violation,No,NaN,hwy 101,16:51:15,17:21:16,1011.0,1041.0,1187.0,-146.0,1,1,United States,United States,"41.3632465, -123.9894411","US-101, United States"
1664,2021-09-29 20:47:00,1900-01-01 20:47:00,white,2_Equipment violation,No,NaN,hwy 101,18:56:14,19:22:59,1136.0,1163.0,1247.0,-84.0,1,1,United States,United States,"41.3632465, -123.9894411","US-101, United States"
1890,2021-11-19 20:31:00,1900-01-01 20:31:00,hispanic/latino(a),2_Equipment violation,No,NaN,hwy 101,16:55:51,17:24:49,1016.0,1045.0,1231.0,-186.0,1,1,United States,United States,"41.3632465, -123.9894411","US-101, United States"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65,2020-10-12 19:56:00,1900-01-01 19:56:00,hispanic/latino(a),NaN,NaN,11.0,woodla north ave a north auburn street,18:36:24,19:03:22,1116.0,1143.0,1196.0,-53.0,1,1,Grass Valley,95945,"39.2212465, -121.0626224","N Auburn St, Grass Valley, CA 95945, USA"
873,2021-03-26 20:44:00,1900-01-01 20:44:00,hispanic/latino(a),2_Equipment violation,NaN,4.0,woodland & auburn,19:28:44,19:55:40,1169.0,1196.0,1244.0,-48.0,1,1,Marin County,94901,"37.9575606, -122.5096168","Auburn St & Woodland Ave, California 94901, USA"
879,2021-03-27 20:17:00,1900-01-01 20:17:00,hispanic/latino(a),2_Equipment violation,NaN,4.0,woodland & auburn,19:29:40,19:56:37,1170.0,1197.0,1217.0,-20.0,1,1,Marin County,94901,"37.9575606, -122.5096168","Auburn St & Woodland Ave, California 94901, USA"
880,2021-03-27 20:34:00,1900-01-01 20:34:00,white,2_Equipment violation,NaN,4.0,woodland & auburn,19:29:40,19:56:37,1170.0,1197.0,1234.0,-37.0,1,1,Marin County,94901,"37.9575606, -122.5096168","Auburn St & Woodland Ave, California 94901, USA"


In [19]:
MARIN_lat_lon = pd.concat([MARIN1,MARIN2,MARIN3,MARIN4])

marin_zips = pd.read_csv('marin_zips - Sheet1.csv')
marin_zips['zip'] = marin_zips['ZIP Code'].str[-5:]
marin_zips = [i for i in marin_zips['zip']]

MARIN_lat_lon = MARIN_lat_lon[MARIN_lat_lon['zip'].notna()]

MARIN_lat_lon.loc[(MARIN_lat_lon['gmap address']=='US-101, United States')|
                  (MARIN_lat_lon['gmap address']=='Bridgeway, Sausalito, CA 94965, USA'),'zip'] = '94965'

MARIN_lat_lon.loc[MARIN_lat_lon['gmap address']=='California\'s 37th Congressional District, CA, USA','zip'] = '94904'
MARIN_lat_lon.loc[MARIN_lat_lon['zip']=='United States','zip'] = '94920'

MARIN_lat_lon.loc[(MARIN_lat_lon['zip'].str.len()<5) |
                  (MARIN_lat_lon['zip'].str.len()>5),'zip'] = MARIN_lat_lon['gmap address'].str[-10:-5]

MARIN_lat_lon.loc[MARIN_lat_lon['zip'].str[0]!='9','zip'] = '94939'


MARIN_lat_lon = MARIN_lat_lon[MARIN_lat_lon['zip'].isin(marin_zips)]


MARIN_lat_lon

,date,time,race,type,race perceived prior,experience,location,sunsettime,dusktime,mins_sunset,mins_dusk,mins_stop_time,mins_before_dusk,itp,dark,neighborhood,zip,coords,gmap address
910,2021-03-31 20:33:00,1900-01-01 20:33:00,white,2_Equipment violation,NaN,21.0,hwy 101,19:33:22,20:00:28,1173.0,1200.0,1233.0,-33.0,1,1,United States,94965,"41.3632465, -123.9894411","US-101, United States"
2019,2021-12-11 19:47:00,1900-01-01 19:47:00,white,2_Equipment violation,No,NaN,hwy 101,16:51:15,17:21:16,1011.0,1041.0,1187.0,-146.0,1,1,United States,94965,"41.3632465, -123.9894411","US-101, United States"
1664,2021-09-29 20:47:00,1900-01-01 20:47:00,white,2_Equipment violation,No,NaN,hwy 101,18:56:14,19:22:59,1136.0,1163.0,1247.0,-84.0,1,1,United States,94965,"41.3632465, -123.9894411","US-101, United States"
1890,2021-11-19 20:31:00,1900-01-01 20:31:00,hispanic/latino(a),2_Equipment violation,No,NaN,hwy 101,16:55:51,17:24:49,1016.0,1045.0,1231.0,-186.0,1,1,United States,94965,"41.3632465, -123.9894411","US-101, United States"
2551,2022-02-27 19:56:00,1900-01-01 19:56:00,white,2_Equipment violation,No,1.0,south knoll road & north knoll road,18:02:20,18:29:21,1082.0,1109.0,1196.0,-87.0,1,1,Bunkie,94939,"30.9542005, -92.18976119999999","S Knoll Ave & N Knoll Ave, Bunkie, LA 71322, USA"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1813,2021-11-05 20:47:00,1900-01-01 20:47:00,hispanic/latino(a),2_Equipment violation,No,7.0,wolfe grade poplar,18:06:59,18:35:03,1087.0,1115.0,1247.0,-132.0,1,1,Marin County,94920,"37.9570896, -122.5357372","Wolfe Grade, California, USA"
873,2021-03-26 20:44:00,1900-01-01 20:44:00,hispanic/latino(a),2_Equipment violation,NaN,4.0,woodland & auburn,19:28:44,19:55:40,1169.0,1196.0,1244.0,-48.0,1,1,Marin County,94901,"37.9575606, -122.5096168","Auburn St & Woodland Ave, California 94901, USA"
879,2021-03-27 20:17:00,1900-01-01 20:17:00,hispanic/latino(a),2_Equipment violation,NaN,4.0,woodland & auburn,19:29:40,19:56:37,1170.0,1197.0,1217.0,-20.0,1,1,Marin County,94901,"37.9575606, -122.5096168","Auburn St & Woodland Ave, California 94901, USA"
880,2021-03-27 20:34:00,1900-01-01 20:34:00,white,2_Equipment violation,NaN,4.0,woodland & auburn,19:29:40,19:56:37,1170.0,1197.0,1234.0,-37.0,1,1,Marin County,94901,"37.9575606, -122.5096168","Auburn St & Woodland Ave, California 94901, USA"


In [21]:
MARIN_lat_lon.isna().sum()

date                      0
time                      0
race                      0
type                     74
race perceived prior    304
experience              102
location                  0
sunsettime                0
dusktime                  0
mins_sunset               0
mins_dusk                 0
mins_stop_time            0
mins_before_dusk          0
itp                       0
dark                      0
neighborhood              0
zip                       0
coords                    0
gmap address              0
dtype: int64

In [169]:
MARIN_lat_lon.to_csv('ALL-MARIN-AB953-DUSK-LOC.csv',index=False)

In [171]:
len(MARIN_lat_lon['zip'].unique())

22

In [75]:
print(pd.read_csv('ALL-OAKLAND_PD-AB953.csv').shape[0])
print(pd.read_csv('ALL-OAKLAND-AB953-DUSK.csv').shape[0])

10817
7446


In [172]:
cd /Users/karatechop/Documents/GitHub/senior-thesis-pretextual-stops/Data/Formatted with YOE

/Users/karatechop/Documents/GitHub/senior-thesis-pretextual-stops/Data/Formatted with YOE


In [173]:
ls

ALL-BPD-AB953-DUSK-LOC.csv    ALL-SDPD-AB953-DUSK.csv
ALL-BPD-AB953-DUSK.csv        ALL-SDPD-AB953.csv
ALL-BPD-AB953.csv             ALL-SFPD-AB953-DUSK-LOC.csv
ALL-MARIN-AB953-DUSK-LOC.csv  ALL-SFPD-AB953-DUSK.csv
ALL-MARIN-AB953-DUSK.csv      ALL-SFPD-AB953.csv
ALL-MARIN-AB953.csv           marin_zips - Sheet1.csv
ALL-OAKLAND-AB953-DUSK.csv    sf_zips.csv
ALL-OAKLAND_PD-AB953.csv


In [174]:
import os
from glob import glob

PATH = os.getcwd()
EXT = "*.csv"

for path, subdir, files in os.walk(PATH):
    for file in glob(os.path.join(path, EXT)):
        if '-LOC' in file:
            print(file,'\n')

/Users/karatechop/Documents/GitHub/senior-thesis-pretextual-stops/Data/Formatted with YOE/ALL-BPD-AB953-DUSK-LOC.csv 

/Users/karatechop/Documents/GitHub/senior-thesis-pretextual-stops/Data/Formatted with YOE/ALL-SFPD-AB953-DUSK-LOC.csv 

/Users/karatechop/Documents/GitHub/senior-thesis-pretextual-stops/Data/Formatted with YOE/ALL-MARIN-AB953-DUSK-LOC.csv 

